# Estimate Elasticity

In this use case, we will examine price elasticity for a regional retailer with 219 locations. The type of retailer isn't important. It could be a convenience store, a restaurant or coffee shop. Whatever the business of the retailer, their data should look basically the same.

The success of a retailer depends on several factors. One, is management and management decisions. Pricing, for example, is 100% controllable by management. Environmental factors surrounding the store are also very important and typically not very controllable. For example, a retailer will typically do better if the people living around the store make a bunch of money.

Our data set in this exercise is a combination of controllable and non-controllable factors. The controllable factor is price. We also have numerous non-controllable environmental factors.

Our goal in this exercise is to better understand the relationship between price, quantity and revenue. We achieve this goal by estimating the price elasticity of demand and the price elasticity of revenue.

# Import data and data transformations

First, let's bring in the python libraries we will need in this exercise.

In [ ]:
!pip install chart_studio  --upgrade
import chart_studio.plotly as py
import plotly.graph_objs as go
import numpy.dual as dual
import plotly as plotly
import sys
import types
import statsmodels.api as sm
import pandas as pd
import numpy as np
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

The next step is to upload the pricing data into object storage. To do this, click on the data icon (it is the one with two 1's and two 0's) in the northeast corner of the Watson Studio interface.

Next, on the "Files" tab click on "Browse" where it says, "Drop your file here or browse your files to add a new file".

From there, navigate to our data set "RETAIL_DATA.csv" and upload it to the cloud.

Once the file is uploaded it should appear in the data tab.



Once the file is loaded into object storage,  from the drop-down menu, select insert credentials.

After this is complete, you should a cell should be created that looks like this.


The next snippet of code will convert the file you just uploaded to object storage into a pandas data frame. Replace the credentials below with the credentials you inserted in the cell above.

In [ ]:
!rm RETAIL_DATA.csv
!wget https://raw.githubusercontent.com/shadgriffin/Pricing_Tutorial/master/RETAIL_DATA.csv


In [ ]:
pd_datax = pd.read_csv("RETAIL_DATA.csv")

df_retail = pd_datax
df_retail.head()

The first few rows of the data set should appear above.

Here is a definition of each field.

STORE_ID - is a unique id specific to each retail outlet

PERCENTAGE_OF_RENTERS is the percentage of households surrounding the store that rent their housing.

PERCENTAGE_OF_CHILDREN is the percentage of households surrounding the store that have children.

AVERAGE_INCOME is the average annual income of the households surrounding the store.

AVERAGE_AGE_IN_YEARS is the average age of the head of household in the vicinity of the retail outlet.

AVERAGE_LENGTH_OF_RESIDENCE is an average of the time individuals surrounding the retail outlet have lived at their current address.

PERCENT_SPEAKING_SPANISH is the percentage of households surrounding the store that speak Spanish

PRICE is the average price across multiple items sold at the retail outlet.

QUANTITY is the number of items sold by the retail outlet in the last year.

REVENUE is the total revenue for the store in the last year.

Our goal in this exercise is to calculate the price elasticity of demand and the price elasticity of revenue. (Please look at the first notebook in this series to understand these terms.) This actually isn't that hard. To estimate an elasticity, you can use a standard ordinary least squares regression and natural log (base e) transformed variables.

The first step is to take the natural log each variable. The cell below does this.  Note the new variables created at the end of the DF.

In [ ]:
df_retail['LN_PRICE'] = np.log((df_retail.PRICE))
df_retail['LN_REVENUE'] = np.log((df_retail.REVENUE))
df_retail['LN_QUANTITY'] = np.log((df_retail.QUANTITY))
df_retail['LN_INCOME'] = np.log((df_retail.AVERAGE_INCOME))
df_retail['LN_AVERAGE_AGE_IN_YEARS'] = np.log((df_retail.AVERAGE_AGE_IN_YEARS))
df_retail['LN_AVERAGE_LENGTH_OF_RESIDENCE'] = np.log((df_retail.AVERAGE_LENGTH_OF_RESIDENCE))
df_retail['LN_PERCENT_SPEAKING_SPANISH'] = np.log((df_retail.PERCENT_SPEAKING_SPANISH))
df_retail['LN_PERCENT_HAVING_CHILDREN'] = np.log((df_retail.PERCENT_HAVING_CHILDREN))
df_retail['LN_PERCENTAGE_OF RENTERS'] = np.log((df_retail.PERCENTAGE_OF_RENTERS))

df_retail.head()


# Build the Model

Now, let's build an ordinary least squares regression using the log transformed variables.

Define your independent and dependent variables with the following code cell. 

Note that ONLY variables that are statistically significant are included.

In [ ]:
independentx = df_retail[['LN_PRICE','LN_PERCENTAGE_OF RENTERS','LN_PERCENT_HAVING_CHILDREN','LN_INCOME',
                          'LN_PERCENT_SPEAKING_SPANISH']]
independent = sm.add_constant(independentx, prepend=False)
dependent=df_retail['LN_QUANTITY']

The next few cells run the OLS regression

In [ ]:
mod = sm.OLS(dependent, independent)

In [ ]:
res = mod.fit()

In [ ]:
print(res.summary())

# Interpreting the OLS Regression Coefficients


So, the price elasticity of demand is -.64. This comes from the ANOVA table above and is the estimated coefficient of LN_PRICE regressed on LN_QUANTITY. 

This means that a 1% increase in price will lower quantity sold by .64%. 

The other coefficients can be interpreted in a similar manner. 

A 1% increase in the average income of people around a store will increase the quantity sold by .55%.



# Creating a Demand Curve

Now, let's make a demand curve. To do this, we will need to build a demand schedule using the predicted quantity at various prices based on our OLS regression model.

First, let's summarize the variables other than price that are significant in the model. We will take the average and then evaluate the relationship between price and quantity when the other variables are at their mean.

In [ ]:
df_retail['chachacha']=1
wookie = df_retail.groupby(['chachacha'])['PERCENTAGE_OF_RENTERS', 'PERCENT_HAVING_CHILDREN','AVERAGE_INCOME','PERCENT_SPEAKING_SPANISH'].mean()

#wookie.reset_index(level=0, inplace=True)
wookie.head()





Next, we will create an array of prices and then convert the list to a pandas data frame.

In [ ]:
#create an array of prices
price = [1.50,1.75,2.0,2.25,2.50,2.75,3.0,3.25,3.50,3.75,4.0,4.25,4.50,4.75,5.0,5.25,5.50,5.75,6.0,6.25,6.50,6.75,
         7.0,7.25,7.5,7.75,8.0,8.25,8.50,8.75,9.0,9.25,9.50,9.75,10.0,10.25,10.50,10.75,11.0,11.25,11.50,11.75,12.0]
df_price=pd.DataFrame(price)
df_price.columns = ['PRICE']
df_price['chachacha']=1


Then, merge the average value for the non-price variables to the prices we constructed above and calculate the log of our variables.

In [ ]:
#join the array of prices to the average values of the other independent variables.
df_price =df_price.merge(wookie, on=['chachacha'], how='inner')

#Create Log Transformed Variables
df_price['LN_PRICE'] = np.log((df_price.PRICE))
df_price['LN_INCOME'] = np.log((df_price.AVERAGE_INCOME))
df_price['LN_PERCENT_SPEAKING_SPANISH'] = np.log((df_price.PERCENT_SPEAKING_SPANISH))
df_price['LN_PERCENT_HAVING_CHILDREN'] = np.log((df_price.PERCENT_HAVING_CHILDREN))
df_price['LN_PERCENTAGE_OF RENTERS'] = np.log((df_price.PERCENTAGE_OF_RENTERS))
df_price['const']=1

df_price.head()




Now, we can use the input variables we manufactured in the previous few lines of code and our model to predict quantity at each price point. The end result will be a demand schedule.

In [ ]:
#create our scoring data set.
scoring= df_price[['LN_PRICE','LN_PERCENTAGE_OF RENTERS','LN_PERCENT_HAVING_CHILDREN','LN_INCOME',
                          'LN_PERCENT_SPEAKING_SPANISH','const']]
#score the scoring data set
ln_q_hat=pd.DataFrame(res.predict(scoring))
#name the columns correctly
ln_q_hat.columns = ['LN_Q_HAT']
#combine ln of price and ln of predicted q into a new data frame
df_ce_demand = pd.concat([scoring['LN_PRICE'], ln_q_hat], axis=1)


#exponentiate the ln variables to get predicted quantity and price
df_ce_demand['Q_HAT']=np.exp(df_ce_demand['LN_Q_HAT'])
df_ce_demand['PRICE']=np.exp(df_ce_demand['LN_PRICE'])

#eliminate the ln variables and make the demand schedule.
df_ce_demand = df_ce_demand[['Q_HAT','PRICE']]

df_ce_demand.head()

Bingo! Now we have a demand schedule and we can plot it as a demand curve.



In [ ]:

trace = go.Scatter(
    x = df_ce_demand['PRICE'],
    y = df_ce_demand['Q_HAT'],
    mode = 'lines'
)



layout = go.Layout(
    title='Demand Curve',
    xaxis=dict(
        title='Price',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Quantity',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)
    
data=[trace]  
fig = go.Figure(data=data, layout=layout)

#plot_url = py.plot(fig, filename='styling-names')
plotly.offline.iplot(fig, filename='shapes-lines')


Wait a minute! That's not linear! Nope, it isn't. This a variation of the demand curves we messed around with in part 1 of our exercise. This is what we call a constant elasticity demand curve. The elasticity is the same, at all points. 

Remember in our earlier discussion that elasticity was different at each price point of the demand curve. With this demand curve, elasticity is the same at all the price points. It still shows the relationship between price and quantity. 

For example, at a price of 6 dollars, our firm can sell about 75,000 units at each store.


If you wanted a linear demand curve, you can definitely get there. You would regress price on quantity (instead of ln of price on ln of quantity). Of course, if you built your model this way, the coefficient wouldn't be an elasticity.

# Estimate Price Elasticity of Revenue

Estimating the price elasticity of revenue follows a similar process. The difference is we will use the natural log of revenue as a dependent variable instead of the natural log of quantity.

In [ ]:
independentx = df_retail[['LN_PRICE','LN_PERCENTAGE_OF RENTERS','LN_PERCENT_HAVING_CHILDREN','LN_INCOME',
                          'LN_PERCENT_SPEAKING_SPANISH']]
independent = sm.add_constant(independentx, prepend=False)
dependent=df_retail['LN_REVENUE']

In [ ]:
mod = sm.OLS(dependent, independent)

In [ ]:
res = mod.fit()

In [ ]:
print(res.summary())

The regression results suggest that price elasticity of revenue is .358. This means that a 1% increase in price will lead to a .358% increase in revenue. In other words, this firm would make more revenue if it increased prices.

There are few important caveats that I should probably mention.

One, this is a point estimate. That is a fancy way of saying that you shouldn't get too crazy. If you increase prices by 1% you probably will increase revenue by .35%. However, if you increase prices by 100%, you probably would not realize a 35% increase in revenue. Baked into the model is an established historical relationship between your customers and your prices. If you do something that is way outside of the historical norm, don't expect the model to be predictive.

Two, it is important to understand this elasticity is an average across all stores. The price elasticity of revenue is .358, on average. There are 291 stores in the data set. Some probably have an elasticity greater than .35. Others probably have an elasticity that is less than .35. In other words, if you increase prices by 3% across the board, on average, you will realize a 1.05% increase in revenue. This is an average. Some stores will realize more than 1.05%. Others will realize less than 1.05%. A 3% increase in prices may even cause some stores to lose revenue.

What if you could tailor the price increase for each store? 

That is, increase prices by an average of 3%, but give some stores a higher bump in prices than others. You could even decrease prices in some stores if it makes sense. Tailoring the price increase to each store based on their specific market, will lead to an even greater increase in revenue. For example, you can raise prices by 3% on average and get an increase in revenue greater than 1.05%.


There are many ways to accomplish this goal. In the third part of this exercise, we will examine a relatively simple and straight forward way to make a market-based pricing decision for each of our 291 stores.
